In [ ]:
#import library
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from sklearn.cluster import KMeans

In [ ]:
df = pd.read_csv('tanggal17_222525_100323.csv',delimiter = ',')
df

In [ ]:
df['date'] = pd.to_datetime(df['date_and_time'], yearfirst = True).dt.date
df['time'] = pd.to_datetime(df['date_and_time']).dt.time
df = df.reindex(columns=['date_and_time', 'date', 'time', 'lat', 'lon', 'speed'])
df = df.drop('date_and_time', axis=1)
df

In [ ]:
df['day'] = pd.to_datetime(df['date']).dt.day_name()
df = df.reindex(columns=['date', 'time', 'day', 'lat', 'lon', 'speed'])
df

In [ ]:
#pull time data and create hour column
df['hour'] = df['time'].astype(str).str[0:2]
 
#convert to numeric    
df['hour'] =  pd.to_numeric(df['hour'])

#convert to integer
df['hour'] = df['hour'].astype('int')

In [ ]:
def jam_sibuk(hour):
    if (hour >= 6 and hour < 9) or (hour >= 16 and hour < 18):
        return 1
    else:
        return 2

In [ ]:
df['jam_sibuk'] = df['hour'].apply(jam_sibuk)
df = df.drop('hour', axis=1)
df

In [ ]:
plt.scatter(df['lon'], df['lat'])
plt.show()

In [ ]:
x = df.iloc[:,3:5]
x

In [ ]:
kmeans = KMeans(n_clusters = 17)
kmeans.fit(x)

In [ ]:
label = kmeans.fit_predict(x)
label

In [ ]:
df['jalur'] = label
df

In [ ]:
df['jalur'].nunique()

In [ ]:
plt.scatter(df['lon'], df['lat'], c= df['jalur'], cmap='rainbow')
plt.show()

In [ ]:
df['avg_speed'] = df.groupby("jalur")["speed"].transform('mean')
df

In [ ]:
def kondisi_kemacetan(avg_speed):
    if avg_speed >= 0 and avg_speed <= 8:
        return 1
    elif avg_speed >= 9 and avg_speed <= 17:
        return 2
    else:
        return 3

In [ ]:
df['kondisi'] = df['avg_speed'].apply(kondisi_kemacetan)
df

In [ ]:
df.to_csv('preprocessing_data.csv', index=False)